## Data Cleaning

In [286]:
import fuzzywuzzy
from fuzzywuzzy import fuzz, process
import chardet
import pandas as pd
import numpy as np
import datetime

In [287]:
# Load up store_income_data.csv
store_df = pd.read_csv("store_income_data.csv")

In [288]:
print(f"There are {len(store_df['country'].unique())} unique countries")

There are 77 unique countries


In [289]:
# Remove trailing white spaces and characters
store_df['country'] = store_df['country'].str.lower().str.strip(" /.")
print(f"There are {len(store_df['country'].unique())} unique countries")
print(store_df['country'].unique())

There are 13 unique countries
['united states' 'britain' 'united kingdom' 'u.k' 'sa' 'america' nan 's.a'
 'england' 'uk' '' 'united states of america' 's. africasouth africa']


2. Note that there should only be three separate countries. Eliminate all variations, so that 'South Africa', 'United Kingdom' and 'United States' are the only three countries.

In [290]:
# Function to replace rows in the provided column of the provided DataFrame
# that match the provided string above the provided ratio with the provided string
def replace_matches_in_column(df, column, string_to_match, min_ratio=80):
    # Get a list of unique strings
    strings = df[column].dropna().unique()  # Drop NaN values to avoid errors

    # Get the top 10 closest matches using fuzz.ratio
    matches = process.extract(string_to_match, strings, limit=10, scorer=fuzz.ratio)

    # Only get matches with a ratio >= min_ratio
    close_matches = [match[0] for match in matches if match[1] >= min_ratio]

    # Find rows that need replacing
    rows_with_matches = df[column].isin(close_matches)

    # Replace values in DataFrame
    df.loc[rows_with_matches, column] = string_to_match

    print(f"Replaced: {close_matches} → {string_to_match}")

In [291]:
replace_matches_in_column(store_df, 'country', "uk")
replace_matches_in_column(store_df, 'country', "sa")

Replaced: ['uk', 'u.k'] → uk
Replaced: ['sa', 's.a'] → sa


In [292]:
print(f"There are {len(store_df['country'].unique())} unique countries")
print(store_df['country'].unique())

There are 11 unique countries
['united states' 'britain' 'united kingdom' 'uk' 'sa' 'america' nan
 'england' '' 'united states of america' 's. africasouth africa']


In [293]:
store_df.country.fillna('other', inplace=True)
store_df.replace('uk', 'united kingdom', inplace=True)
store_df.replace('britain', 'united kingdom', inplace=True)
store_df.replace('england', 'united kingdom', inplace=True)
store_df.replace('united states of america', 'united states', inplace=True)
store_df.replace('america', 'united states', inplace=True)
store_df.replace('sa', 'south africa', inplace=True)
store_df.replace('s. africasouth africa', 'south africa', inplace=True)
store_df.replace('', 'other', inplace=True)

C:\Users\shazz\AppData\Local\Temp\ipykernel_15068\608647543.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  store_df.country.fillna('other', inplace=True)


In [294]:
print(f"There are {len(store_df['country'].unique())} unique countries")
print(store_df['country'].unique())

There are 4 unique countries
['united states' 'united kingdom' 'south africa' 'other']


In [295]:
# Convert to datetime using the correct format
store_df['date_measured'] = pd.to_datetime(store_df['date_measured'], format='%d-%m-%Y')


# Calculate days ago
today = pd.Timestamp(datetime.date.today())
store_df['days_ago'] = (today - store_df['date_measured']).dt.days

print(store_df[['date_measured', 'days_ago']].head())

  date_measured  days_ago
0    2006-02-04      6947
1    2006-01-04      6978
2    2003-09-12      7823
3    2006-05-08      6854
4    1973-01-21     19014
